# Módulo 2
**Desarrollo de proyectos de análisis de datos  IN1002B**

### Detección y corrección de inconsistencias.



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from scipy import stats

La base de datos **landslide** contiene información de avalanchas ocurridas en todo el mundo.

En esta sesión, exploraremos la base de datos haciendo énfasis sobre que día, mes, año y hora ocurrieron los eventos. Así mismo, realizaremos actividades de limpieza y preparación.

In [3]:
data = pd.read_csv('/content/drive/MyDrive/IN1002B_204/landslide.csv')  # Binder -> Data/landslide.csv

In [4]:
# info()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1693 entries, 0 to 1692
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1693 non-null   int64  
 1   date                  1690 non-null   object 
 2   time                  629 non-null    object 
 3   continent_code        164 non-null    object 
 4   country_name          1693 non-null   object 
 5   country_code          1693 non-null   object 
 6   state/province        1692 non-null   object 
 7   population            1693 non-null   int64  
 8   city/town             1689 non-null   object 
 9   distance              1692 non-null   float64
 10  location_description  551 non-null    object 
 11  latitude              1692 non-null   float64
 12  longitude             1692 non-null   float64
 13  geolocation           1692 non-null   object 
 14  hazard_type           1693 non-null   object 
 15  landslide_type       

In [5]:
#shape
data.shape

(1693, 23)

In [6]:
# head()
data.head(1)

,id,date,time,continent_code,country_name,country_code,state/province,population,city/town,distance,...,geolocation,hazard_type,landslide_type,landslide_size,trigger,storm_name,injuries,fatalities,source_name,source_link
0,34,3/2/07,Night,NaN,United States,US,Virginia,16000,Cherry Hill,3.40765,...,"(38.600900000000003, -77.268199999999993)",Landslide,Landslide,Small,Rain,NaN,NaN,NaN,NBC 4 news,http://www.nbc4.com/news/11186871/detail.html


## **Ajustes previos**

Eliminen las columnas que consideren no relevantes, o bien que no puedan ser sujetas a ajustes de reemplazo:



In [7]:
data.isnull().sum()

,0
id,0
date,3
time,1064
continent_code,1529
country_name,0
country_code,0
state/province,1
population,0
city/town,4
distance,1


In [8]:
# Seleccionar las columnas a eliminar (menos la de time, la usaremos despues)
columnas = ['continent_code','location_description', 'storm_name']

In [9]:
# Aplicar la función drop()

data.drop(columns = columnas, inplace = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1693 entries, 0 to 1692
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              1693 non-null   int64  
 1   date            1690 non-null   object 
 2   time            629 non-null    object 
 3   country_name    1693 non-null   object 
 4   country_code    1693 non-null   object 
 5   state/province  1692 non-null   object 
 6   population      1693 non-null   int64  
 7   city/town       1689 non-null   object 
 8   distance        1692 non-null   float64
 9   latitude        1692 non-null   float64
 10  longitude       1692 non-null   float64
 11  geolocation     1692 non-null   object 
 12  hazard_type     1693 non-null   object 
 13  landslide_type  1692 non-null   object 
 14  landslide_size  1692 non-null   object 
 15  trigger         1691 non-null   object 
 16  injuries        515 non-null    float64
 17  fatalities      1446 non-null   f

## **Fecha y hora**

### Fechas
Cuando queremos trabajar con fechas, podemos usar la funcion de ```to_datetime()```, lo unico que debemos revisar es el orden en el que se cargaron los datos, es decir si se encuentra:
* dd/mm/aaaa
* mm-dd-aaaa
* aaaa dd mm
* etc

En el mismo orden y con los mismos caracteres especiales, le especificaremos a la función que queremos convertir y agregaremos el simbolo de $%$.

* ```%d/%m/%y``` - ```%d/%m/%Y```
* ```%m-%d-%y``` - ```%m-%d-%Y```
* ```%Y %d %m```

Realicemos el ajuste en la columna ```date```

- Impriman un ```head()``` de la columna **date** para ver el formato de la fecha:


In [10]:
# head()
data.head(2)

,id,date,time,country_name,country_code,state/province,population,city/town,distance,latitude,longitude,geolocation,hazard_type,landslide_type,landslide_size,trigger,injuries,fatalities,source_name,source_link
0,34,3/2/07,Night,United States,US,Virginia,16000,Cherry Hill,3.40765,38.6009,-77.2682,"(38.600900000000003, -77.268199999999993)",Landslide,Landslide,Small,Rain,NaN,NaN,NBC 4 news,http://www.nbc4.com/news/11186871/detail.html
1,42,3/22/07,NaN,United States,US,Ohio,17288,New Philadelphia,3.33522,40.5175,-81.4305,"(40.517499999999998, -81.430499999999995)",Landslide,Landslide,Small,Rain,NaN,NaN,Canton Rep.com,http://www.cantonrep.com/index.php?ID=345054&C...


In [12]:
data['date'] = pd.to_datetime(data['date'], format= '%m/%d/%y')

In [13]:
#info()
#¿qué tipo de dato es nuestra columna date ahora?
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1693 entries, 0 to 1692
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              1693 non-null   int64         
 1   date            1690 non-null   datetime64[ns]
 2   time            629 non-null    object        
 3   country_name    1693 non-null   object        
 4   country_code    1693 non-null   object        
 5   state/province  1692 non-null   object        
 6   population      1693 non-null   int64         
 7   city/town       1689 non-null   object        
 8   distance        1692 non-null   float64       
 9   latitude        1692 non-null   float64       
 10  longitude       1692 non-null   float64       
 11  geolocation     1692 non-null   object        
 12  hazard_type     1693 non-null   object        
 13  landslide_type  1692 non-null   object        
 14  landslide_size  1692 non-null   object        
 15  trig

### Horas

Ahora realizaremos el ajuste en la columna de ```time```.


In [14]:
#unique()
data.time.unique()

array(['Night', nan, '19:30:00', 'Early morning', '1:41:00', 'Afternoon',
       '11:30:00', 'Morning', '23:25:00', '18:00:00', '22:00:00',
       '17:36:00', 'Overnight', '16:30:00', '13:40:00', '15:00:00',
       '7:00:00', '19:00:00', '22:00', '0:00:00', '12:00', '6:45:00',
       '18:15:00', '2:00:00', 'Late morning', '23:30:00', '11:00:00',
       '7:19:00', 'Evening', '21:00:00', '5:00:00', '10:00:00',
       '17:00:00', '1:30:00', '12:00:00', '5:30:00', '6:00:00',
       'Late evening', '8:00:00', '14:30:00', '15:45:00', '9:30:00',
       '18:46:00', '2:30:00', '15:30:00', '13:45:00', '15:40:00',
       '14:10:00', '4:30:00', '18:53:00', '11:45:00', 'Late night',
       '23:00:00', '13:00:00', '3:00:00', '4:00:00', '20:00:00', '13:00',
       '0:15:00', '0:30:00', '10:30:00', 'Before dawn', '9:00:00',
       '7:30:00', '14:00:00', '1:00:00', '12:30:00', '21:30:00', '12:15',
       '1:52:00', 'Late afternoon', '9:50:00', '5:45:00', '3:30:00',
       '21:40:00', '20:40:00', '15:00

Estos son los valores que requerían un reemplazo.

Los números fueron colocados sin un criterio en especifico, pueden ajustarlos si lo desean.

In [15]:
data['time'].replace({'Night': '22:00','Early morning': '07:00', 'Afternoon':'13:00','Late morning': '10:00',
                    'Before dawn':'18:00', 'Late afternoon': '16:00', 'evening':'21:00', '****':0 , 'Unknown': 0, 'overnight':'23:00',
                     'Evening':'21:00','Overnight':'01:00', 'Morning':'08:00', ' ':0, 'Late evening': '21:00', 'Late night': '23:00'}, inplace = True)

In [16]:
# unique()
data.time.unique()

array(['22:00', nan, '19:30:00', '07:00', '1:41:00', '13:00', '11:30:00',
       '08:00', '23:25:00', '18:00:00', '22:00:00', '17:36:00', '01:00',
       '16:30:00', '13:40:00', '15:00:00', '7:00:00', '19:00:00',
       '0:00:00', '12:00', '6:45:00', '18:15:00', '2:00:00', '10:00',
       '23:30:00', '11:00:00', '7:19:00', '21:00', '21:00:00', '5:00:00',
       '10:00:00', '17:00:00', '1:30:00', '12:00:00', '5:30:00',
       '6:00:00', '8:00:00', '14:30:00', '15:45:00', '9:30:00',
       '18:46:00', '2:30:00', '15:30:00', '13:45:00', '15:40:00',
       '14:10:00', '4:30:00', '18:53:00', '11:45:00', '23:00', '23:00:00',
       '13:00:00', '3:00:00', '4:00:00', '20:00:00', '0:15:00', '0:30:00',
       '10:30:00', '18:00', '9:00:00', '7:30:00', '14:00:00', '1:00:00',
       '12:30:00', '21:30:00', '12:15', '1:52:00', '16:00', '9:50:00',
       '5:45:00', '3:30:00', '21:40:00', '20:40:00', '15:00', '17:45:00',
       0, '11:50:00', '15:32', '9:40:00', '1:13', '3:20:00', '20:45:00',
       

Como pudieron notar, no existe un formato de **time** en esta ocasión.

Lo bueno es que dentro de ```pd.to_datetime()``` existe una opción llamada **mixed** que infiere el estilo y acomoda la hora.


In [17]:
data['time'] = pd.to_datetime(data['time'], format='mixed')

In [18]:
# imrpimir la columna time
data.time

,time
0,2024-09-24 22:00:00
1,NaT
2,NaT
3,NaT
4,NaT
...,...
1688,NaT
1689,2024-09-24 00:00:00
1690,NaT
1691,2024-09-24 21:06:00


Ahora, con la función ```isnull()``` veamos cuantos valores faltantes tenemos:

In [19]:
# isnull()
data.isnull().sum()

,0
id,0
date,3
time,1064
country_name,0
country_code,0
state/province,1
population,0
city/town,4
distance,1
latitude,1


Considerando esta situación, ¿que tipo de reemplazo consideran apropiado? ¿cuál debería ser la lógica?

Hagamos los pasos necesarios para construir una función. Revisemos cuál es la horá más frecuente:

In [20]:
data.time.value_counts()

,count
time,
2024-09-24 22:00:00,100
2024-09-24 08:00:00,95
2024-09-24 13:00:00,65
2024-09-24 07:00:00,38
2024-09-24 21:00:00,27
...,...
2024-09-24 15:45:00,1
2024-09-24 16:58:00,1
2024-09-24 20:20:00,1


Debido a que no hay una **tendencia** significativa, utilizaremos otra técnica:

### ```groupby()```

Esta función de pandas nos permite agrupar los datos respecto a un conjunto de categorías, donde a su vez les aplicamos una función adicional (media, mediana, moda, etc.)

1. Obtener las modas por país

In [21]:
#Veamos cuales es la moda respecto a la hora en que ocurrieron los eventos por país

modas = data.groupby('country_name')['time'].agg(
    lambda x: x.mode()[0] if not x.mode().empty else None
)
print(modas)

country_name
Barbados                                           NaT
Belize                             2024-09-24 10:00:00
Bermuda                            2024-09-24 06:45:00
Brazil                             2024-09-24 00:30:00
Canada                             2024-09-24 06:00:00
Colombia                           2024-09-24 22:00:00
Costa Rica                         2024-09-24 13:00:00
Cuba                                               NaT
Dominica                           2024-09-24 02:00:00
Dominican Republic                 2024-09-24 22:00:00
Ecuador                            2024-09-24 22:00:00
El Salvador                        2024-09-24 22:00:00
Grenada                            2024-09-24 03:00:00
Guatemala                          2024-09-24 22:00:00
Haiti                              1970-01-01 00:00:00
Honduras                           2024-09-24 08:00:00
Jamaica                            2024-09-24 00:15:00
Mexico                             2024-09-24 08:00:

2. Reemplzar los valores tipo NaT por una de las modas más recurrentes: ```2024-01-01 10:00:00```

In [23]:
modas = modas.fillna(pd.Timestamp('2024-01-01 10:00:00'))
modas

,time
country_name,
Barbados,2024-01-01 10:00:00
Belize,2024-09-24 10:00:00
Bermuda,2024-09-24 06:45:00
Brazil,2024-09-24 00:30:00
Canada,2024-09-24 06:00:00
Colombia,2024-09-24 22:00:00
Costa Rica,2024-09-24 13:00:00
Cuba,2024-01-01 10:00:00
Dominica,2024-09-24 02:00:00


3. Crear una función de reemplzo

In [24]:
def calcular_moda_y_reemplazar_nan(df, col_x, col_y, hora):
    # 1. Agrupar por la columna y calcular la moda de la columna x
    modas = df.groupby(col_y)[col_x].agg(
        lambda x: x.mode()[0] if not x.mode().empty else None
    )

    # 2. Reemplazar cualquier NaT que no se haya reemplazado con un valor específico
    df[col_x] = df[col_x].fillna(pd.Timestamp(hora))

    # 3. Reemplazar los NaT en el DataFrame original con las modas
    for valor, moda in modas.items():
        df.loc[(df[col_y] == valor) & (df[col_x].isna()), col_x] = moda



    return df



df = calcular_moda_y_reemplazar_nan(data, 'time', 'country_name', '2024-09-20 10:00:00')

1) ```df.groupby(col_y)```: Agrupa el DataFrame por la columna ```col_y```, es decir, por los nombres de los países.
2) ```[col_x]```: Selecciona la columna ```col_x``` (donde se encuentran los valores que queremos reemplazar).
3) ```.agg(...)```: Aplica una función de agregación a cada grupo.

 ```lambda x: x.mode()[0] if not x.mode().empty else None:```

- Calcula la moda de los valores en cada grupo.
- ```x.mode()``` devuelve una Serie con los valores más frecuentes.

Si hay una moda, toma el primer valor ([0]), y si no hay moda, devuelve None.

-----------------------------------------------------------------------------------------------------------------------------------------------------------

Ahora, impriman un ```value_counts()``` de la columna **time** para ver como quedo:

In [25]:
# value_counts()
data.time.value_counts()

,count
time,
2024-09-20 10:00:00,1064
2024-09-24 22:00:00,100
2024-09-24 08:00:00,95
2024-09-24 13:00:00,65
2024-09-24 07:00:00,38
...,...
2024-09-24 15:15:00,1
2024-09-24 19:45:00,1
2024-09-24 04:55:00,1


Ahora hagamos un ```dropna()``` en toda la base de datos:

In [26]:
df.dropna(inplace = True)
df.shape

(486, 20)

## **Análisis**

Vamos a extraer la información de las columnas de **date** y **time** para responder algunas preguntas de análisis:

### Més

In [27]:
# creamos la columna mes
df['month'] = df['date'].dt.month

### Año

In [28]:
df['year'] = df['date'].dt.year

In [30]:
df.year

,year
168,2008
217,2009
252,2010
346,2010
520,2010
...,...
1687,2015
1689,2016
1690,2016
1691,2016


*Las siguientes preguntas aparecerán en su actividad 2.2*

a) ¿En que mes se detectaron más avalanchas? (pueden utilizar: ```.value_counts(normalize = True)```)

b) ¿En que país se detectaron más muertes promedio? (pueden utilizar: ```groupby()[].agg['mean']```).

c) Considerando solo Estados Unidos, ¿en que estado las avalanchas fueron más largas (*distance*) en promedio)?

Pueden consultar el siguiente <a href="https://www.weforum.org/agenda/2016/04/does-this-explain-why-some-landslides-travel-much-further-than-others/">artículo</a> para complementar su respuesta.



In [ ]:
# a)

In [ ]:
# b)

In [ ]:
# c)